In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import os

In [21]:
country_list = []
for dirname, _, filenames in os.walk('./All_by_country'):
    for filename in filenames:
        txt = os.path.join(dirname, filename)
        txt = txt.replace('./All_by_country/', '')
        txt = txt.replace('_all.csv', '')
        if txt == '.DS_Store':
            continue
        country_list.append(txt)
        print(txt) 

Burundi
Honduras
El Salvador
Rwanda
México
Bolivia
Indonesia
Brazil
Colombia
Ethiopia
Perú
Costa Rica
Guatemala
Nicaragua
Ecuador


## Drop NaNs

In [22]:
# drop if 'Coffee_characteristics' is nan
def drop_nan(df):
    df = df.dropna(subset=['Coffee_characteristics'])
    return df

## Rank

In [23]:
def rank(df):
    # if its type already is int or float, return it
    if df['Rank'].dtype == int or df['Rank'].dtype == float:
        return df
    df['Rank'] = df['Rank'].str.replace('a', '')
    df['Rank'] = df['Rank'].str.replace('A', '')
    df['Rank'] = df['Rank'].str.replace('b', '')
    df['Rank'] = df['Rank'].str.replace('B', '')
    df['Rank'] = df['Rank'].str.replace('c', '')
    df['Rank'] = df['Rank'].str.replace('C', '')
    df['Rank'] = df['Rank'].astype(int)
    return df

## Score

In [24]:
def score(df):
    if df['COE_score'].dtype != float:
        df['COE_score'] = df['COE_score'].str.replace(',', '.')
        df['COE_score'] = df['COE_score'].astype(float)
    return df

In [25]:
def plot_score(df, country):
    sns.set_style('whitegrid')
    sns.distplot(df['COE_score'], bins=30, kde=False)
    plt.title(f'Distribution of COE scores in {country}')
    plt.xlabel('COE score')
    plt.ylabel('Number of coffees')
    plt.show()

## High bids

In [26]:
def high_bid(df):
    # clean the price data, remove '$', '/lb' and ','
    df['High_bid'] = df['High_bid'].str.replace('$', '')
    df['High_bid'] = df['High_bid'].str.replace('/lb', '')
    df['High_bid'] = df['High_bid'].str.replace('US', '')
    # df['High_bid'] = df['High_bid'].str.replace(',', '')
    df['High_bid'] = df['High_bid'].astype(float)
    return df

In [27]:
def plot_high_bid(df, country):
    sns.set_style('whitegrid')
    sns.distplot(df['High_bid'], bins=30, kde=False)
    plt.title(f'Distribution of High bids in {country}')
    plt.xlabel('High bid')
    plt.ylabel('Number of coffees')
    plt.show()

## Total value

In [28]:
def total_value(df):
    df['Total_value'] = df['Total_value'].str.replace('$', '')
    df['Total_value'] = df['Total_value'].str.replace(',', '')
    df['Total_value'] = df['Total_value'].str.replace('US', '')
    df['Total_value'] = df['Total_value'].str.replace('$.', '')
    df['Total_value'] = df['Total_value'].str.replace('\\xa0', '')
    df['Total_value'] = df['Total_value'].str.strip()
    df['Total_value'] = df['Total_value'].astype(float)
    return df

## Altitude

In [29]:
def altitude(df):
    # if its type already is int or float, return it
    if df['Altitude'].dtype == int or df['Altitude'].dtype == float:
        return df
    to_remove = [' ','m.a.s.l.','m.a.s.l','m','masl','asl','ass','ts','mts',',',' ','asl','ass']
    # if contain '-', keep the latter one
    df['Altitude'] = df['Altitude'].str.split('-').str[-1]
    df['Altitude'] = df['Altitude'].str.strip()
    for r in to_remove:
        df['Altitude'] = df['Altitude'].str.replace(r, '')
    # if got '', let it be nan
    df['Altitude'] = df['Altitude'].replace('', np.nan)
    # if it's not nan, convert to float
    df['Altitude'] = pd.to_numeric(df['Altitude'], errors='coerce')
    # if it's over 3000 convert from feet to meters
    df.loc[df['Altitude'] > 3500, 'Altitude'] = df.loc[df['Altitude'] > 3500, 'Altitude'] * 0.3048

    return df

In [30]:
def plot_altitude(df, country):
    sns.set_style('whitegrid')
    sns.distplot(df['Altitude'], bins=30, kde=False)
    plt.title(f'Distribution of Altitude of {country}')
    plt.xlabel('Altitude')
    plt.ylabel('Number of coffees')
    plt.show()

## Flavor

In [31]:
def flavor(df):
    df['Coffee_characteristics'].fillna('', inplace=True)
    df['Flavor'].fillna('', inplace=True)
    df['Acidity'].fillna('', inplace=True)

    df['Coffee_characteristics'].astype(str)
    df['Coffee_characteristics'] = df['Coffee_characteristics'].str.lower()
    df['Flavor'] = df['Flavor'].astype(str).str.lower()
    df['Acidity'] = df['Acidity'].astype(str).str.lower()

    # remove '(' , ')', and numbers
    to_remove = ['(', ')', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']
    for r in to_remove:
        df['Coffee_characteristics'] = df['Coffee_characteristics'].str.replace(r, '')
        df['Flavor'] = df['Flavor'].str.replace(r, '')
        df['Acidity'] = df['Acidity'].str.replace(r, '')

    # there are some characteristics a coffee good to have: Clean Cup, Balenced Cup, Transparent Cup, Creamy Body, Big Body, Long Aftertaste,
    clean_words = ['clean', 'clear']
    balanced_words = ['balance', 'balanced', 'hamonious','consistent']
    transparent_words = ['transparent']
    creamy_words = ['creamy', 'cream', 'buttery', 'butter']
    big_words = ['big', 'full', 'heavy', 'thick', 'dense']
    long_words = ['long', 'lasting', 'persistent', 'prolong', 'lingering', 'silky']
    # Convert lists to regular expressions
    clean_regex = '|'.join(clean_words)
    balanced_regex = '|'.join(balanced_words)
    transparent_regex = '|'.join(transparent_words)
    creamy_regex = '|'.join(creamy_words)
    big_regex = '|'.join(big_words)
    long_regex = '|'.join(long_words)
    # if the coffee has the characteristics, then 1 in the col, otherwise 0
    df['Clean_and_clear'] = df['Coffee_characteristics'].str.contains(clean_regex).astype(int)
    df['Balance_cup'] = df['Coffee_characteristics'].str.contains(balanced_regex).astype(int)
    df['Transparent_cup'] = df['Coffee_characteristics'].str.contains(transparent_regex).astype(int)
    df['Creamy_body'] = df['Coffee_characteristics'].str.contains(creamy_regex).astype(int)
    df['Big_body'] = df['Coffee_characteristics'].str.contains(big_regex).astype(int)
    df['Long_aftertaste'] = df['Coffee_characteristics'].str.contains(long_regex).astype(int)

    # divide into Roasted flavor, Spices flavor, Nutty/cocoa flavor, Sweet flavor, Floral flavor, Fruity flavor, and Green Vegetative flavor
    roast_words = ['roasted', 'roasty', 'roast', 'burnt', 'burnt', 'smoky', 'cereal', 'ashy', 'tabacco', 'grain']
    spices_words = ['spices', 'spice', 'cinnamon', 'clove', 'pepper', 'anise', 'nutmeg']
    cocoa_words = ['cocoa', 'cacao', 'chocolate', 'chocolaty', 'chocolatey','peanut', 'almond', 'hazelnut', 'nutty', 'nut', 'walnut', 'marzipan', 'praline', 'caramel', 'butterscotch', 'toffee']
    sweet_words = ['sweet', 'sugar', 'honey', 'maple','caramel','caramelized', 'syrup', 'malt', 'sugarcane']
    floral_words = ['floral', 'flower', 'rose', 'jasmine', 'lavender', 'lilac', 'hibiscus', 'chamomile', 'honeysuckle', 'blossom']
    fruity_words = ['raisin','fruit', 'fruity', 'berry', 'blueberry', 'blackberry', 'raspberry', 'strawberry', 'cranberry', 'cherry', 'redcurrant','grape','apple','peach','mango','grapefruit','lemon','lime','orange','tangerine','apricot','plum','prune','raisin','papaya','pineapple','banana','coconut','passionfruit','lychee','melon','apricot']
    green_vege_words = ['green', 'vegetative', 'vegetable', 'herb', 'herbal', 'grass', 'grassy', 'olive', 'seeds']
    wine_words = ['wine', 'winey', 'fermented', 'ferment', 'fermentation', 'wiskey', 'rum']
    # Convert lists to regular expressions
    roast_regex = '|'.join(roast_words)
    spices_regex = '|'.join(spices_words)
    cocoa_regex = '|'.join(cocoa_words)
    sweet_regex = '|'.join(sweet_words)
    floral_regex = '|'.join(floral_words)
    fruity_regex = '|'.join(fruity_words)
    green_vege_regex = '|'.join(green_vege_words)
    wine_regex = '|'.join(wine_words)
    # if the coffee has the characteristics, then +1 in the col, otherwise +0
    df['Roasted_flavor'] = df['Coffee_characteristics'].str.count(roast_regex).astype(int)
    df['Spices_flavor'] = df['Coffee_characteristics'].str.count(spices_regex).astype(int)
    df['NuttyCocoa_flavor'] = df['Coffee_characteristics'].str.count(cocoa_regex).astype(int)
    df['Sweet_flavor'] = df['Coffee_characteristics'].str.count(sweet_regex).astype(int)
    df['Floral_flavor'] = df['Coffee_characteristics'].str.count(floral_regex).astype(int)
    df['Fruity_flavor'] = df['Coffee_characteristics'].str.count(fruity_regex).astype(int)
    df['GreenVegetative_flavor'] = df['Coffee_characteristics'].str.count(green_vege_regex).astype(int)
    df['Winey_flavor'] = df['Coffee_characteristics'].str.count(wine_regex).astype(int)

    # do the same thing for flavor 
    df['Roasted_flavor'] += df['Flavor'].str.count(roast_regex).astype(int)
    df['Spices_flavor'] += df['Flavor'].str.count(spices_regex).astype(int)
    df['NuttyCocoa_flavor'] += df['Flavor'].str.count(cocoa_regex).astype(int)
    df['Sweet_flavor'] += df['Flavor'].str.count(sweet_regex).astype(int)
    df['Floral_flavor'] += df['Flavor'].str.count(floral_regex).astype(int)
    df['Fruity_flavor'] += df['Flavor'].str.count(fruity_regex).astype(int)
    df['GreenVegetative_flavor'] += df['Flavor'].str.count(green_vege_regex).astype(int)
    df['Winey_flavor'] += df['Flavor'].str.count(wine_regex).astype(int)


    # divide acidity into: Malic, Citric, Tartaric, Lactic, Complex
    malic_words = ['malic', 'apple', 'green apple']
    citric_words = ['citric', 'lemon', 'lime', 'citrus', 'mandarin']
    tartaric_words = ['tartaric', 'grape', 'grapefruit']
    lactic_words = ['lactic', 'milk', 'milk chocolate']
    complex_words = ['complex', 'complexity']
    # Convert lists to regular expressions
    malic_regex = '|'.join(malic_words)
    citric_regex = '|'.join(citric_words)
    tartaric_regex = '|'.join(tartaric_words)
    lactic_regex = '|'.join(lactic_words)
    complex_regex = '|'.join(complex_words)
    # if the coffee has the characteristics, then +1 in the col, otherwise +0
    df['Malic_acidity'] = df['Acidity'].str.count(malic_regex).astype(int)
    df['Citric_acidity'] = df['Acidity'].str.count(citric_regex).astype(int)
    df['Tartaric_acidity'] = df['Acidity'].str.count(tartaric_regex).astype(int)
    df['Lactic_acidity'] = df['Acidity'].str.count(lactic_regex).astype(int)
    df['Complex_acidity'] = df['Acidity'].str.count(complex_regex).astype(int)
    df['Winey_flavor'] += df['Acidity'].str.count(wine_regex).astype(int)
    return df

## Process

In [32]:
def process(df):
    main_process = ['washed', 'natural', 'honey', 'anaerobic', 'wet hulled']
    df['Process'] = df['Process'].str.lower()
    df['Process'].fillna('', inplace=True)
    df['Process'] = df['Process'].str.replace('coffee', '')
    df['Process'] = df['Process'].str.replace(',', '')
    # if the first letter or the last letter is ' ', remove it
    df['Process'] = df['Process'].str.strip()
    ## anaerobic
    # replace 'anaerobico', 'natural anaerobic', 'anaerobic washed', 'anaerobic honey', 'washed anaerobic', 'honey anearobic' with 'anaerobic'
    to_replace = ['anaerobico', 'natural anaerobic', 'anaerobic washed', 'anaerobic honey', 'washed anaerobic', 'honey anaerobic']
    for r in to_replace:
        df['Process'] = df['Process'].str.replace(r, 'anaerobic')
    ## wet uulled
    # if contais 'depulped' or 'wet uulled', replace it with 'depulped'
    df.loc[df['Process'].str.contains('depulped'), 'Process'] = 'wet hulled'
    df.loc[df['Process'].str.contains('wet hulled'), 'Process'] = 'wet hulled'
    ## others
    # if the process is not in the main process, replace it with 'others'
    df.loc[~df['Process'].isin(main_process), 'Process'] = 'others_process'
    
    return df

## Variety

In [33]:
correcting_dic = {
        'gesha': 'Geisha', 
        'Gesha': 'Geisha',
        'bourbón': 'Bourbon',
        'Bourbón': 'Bourbon',
        'catuaí': 'Catuai',
        'Catuaí': 'Catuai',
        'sl34': 'sl-34',
        'sl 34': 'sl-34',
        'sl28': 'sl-28',
        'sl 28': 'sl-28',
        'yellow bourbon': 'Yellow_Bourbon',
        'red bourbon': 'Red_Bourbon',
        'pink bourbon': 'Pink_Bourbon'}

varieties_wanted = ['geisha', 'pacamara', 'catuai', 'caturra', 'bourbon', 'typica', 'sl-28', 'sl-34', 'colombia', '74112', '74110', '74158']

def variety(df):

    # variety category
    # print('Variety category')
    # print(df['Variety'].unique())

    # correct spelling
    df['Variety'] = df['Variety'].astype(str)

    for key, value in correcting_dic.items():
        df['Variety'] = df['Variety'].str.replace(key, value)
    
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('á', 'a'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('ã', 'a'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('é', 'e'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('í', 'i'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('ó', 'o'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('ú', 'u'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('ñ', 'n'))

    df['Variety'] = df['Variety'].apply(lambda x: x.replace('Catucai', 'Catuai'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('Cauai', 'Catuai'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('Catui', 'Catuai'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('Cataui', 'Catuai'))

    df['Variety'] = df['Variety'].apply(lambda x: x.replace('Tipica', 'Typica'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('TIPICA', 'Typica'))

    df['Variety'] = df['Variety'].apply(lambda x: x.replace('GESHA', 'Geisha'))

    df['Variety'] = df['Variety'].apply(lambda x: x.replace('Borbon', 'Bourbon'))

    df['Variety'] = df['Variety'].apply(lambda x: x.replace('SL-28', 'sl-28'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('SL28', 'sl-28'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('SL 28', 'sl-28'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('SL-34', 'sl-34'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('SL34', 'sl-34'))
    df['Variety'] = df['Variety'].apply(lambda x: x.replace('SL 34', 'sl-34'))

    # print('Variety category after correcting')
    # print(df['Variety'].unique())

    # wanted varieties dummy col
    for v in varieties_wanted:
        df[v] = df['Variety'].apply(lambda x: 1 if v in x or v.capitalize() in x or v.upper() in x else 0)
    
    # mixed_variety dummy col
    df['mixed_variety'] = df['geisha'] + df['pacamara'] + df['catuai'] + df['caturra'] + df['bourbon'] + df['typica'] + df['sl-28'] + df['sl-34'] + df['colombia'] + df['74112'] + df['74110'] + df['74158']
    df['mixed_variety'] = df['mixed_variety'].apply(lambda x: 1 if x>1 else 0)
    
    return df

In [34]:
def plot_variety(df, country):
    sns.set_style('whitegrid')
    sns.countplot(x='Process', data=df)
    plt.title(f'Number of coffees of each process in {country}')
    plt.xlabel('Process')
    plt.ylabel('Number of coffees')
    plt.show()

## To CSV

In [35]:
def save_to_csv(df, country):
    cols_we_want = ['Rank', 'Country', 'Year','COE_score', 'Farm', 'High_bid', 'Total_value', 'Company',
       'Farmer/Rep.', 'Altitude', 'City', 'Region','Process',
       'Clean_and_clear', 'Balance_cup', 'Transparent_cup', 'Creamy_body', 'Big_body', 'Long_aftertaste', 
       'Roasted_flavor', 'Spices_flavor', 'NuttyCocoa_flavor', 'Sweet_flavor', 'Floral_flavor', 'Fruity_flavor', 'GreenVegetative_flavor', 'Winey_flavor',
       'Malic_acidity', 'Citric_acidity', 'Tartaric_acidity', 'Lactic_acidity','Complex_acidity', 
       'geisha', 'pacamara', 'catuai', 'caturra', 'bourbon', 'typica', 'sl-28', 'sl-34', 'colombia', '74112', '74110', '74158', 'mixed_variety']
    df = df[cols_we_want]
    df.to_csv(f'All_cleaned_v2/{country}_cleaned_v2.csv')

-----

In [36]:
for c in country_list:
    country = c
    if country == 'Bolivia':
        continue
    print(country)
    df = pd.read_csv(f'All_by_country/{country}_all.csv')
    df = drop_nan(df)
    df = rank(df)
    df = score(df)
    df = high_bid(df)
    df = total_value(df)
    df = altitude(df)
    df = flavor(df)
    df = process(df)
    df = variety(df)
    # plot_score(df, country)
    # plot_altitude(df, country)
    # plot_high_bid(df, country)
    # plot_variety(df, country)
    save_to_csv(df, country)
    # print('done')
print('all done')

Burundi
Honduras
El Salvador
Rwanda
México


Indonesia
Brazil
Colombia
Ethiopia
Perú
Costa Rica
Guatemala
Nicaragua
Ecuador
all done


## note
- Indonesia - total value: manually change one data that returns : ```could not convert string to float: '.\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 11964.53'```
- Ethiopia - Year: some are missing, manually fill with 2020
- Costa Rica - Year: some are missing, manually fill with 2023
- Guatemala - Year: one is missing, manually fill with 2019
- Honduras - Year: one is missing, manually fill with 2023
